In [91]:
from collections import defaultdict
import numpy as np
from matplotlib import pyplot as plt
import msvcrt
import pickle
import itertools

class Program:
    def __init__(self, code):
        self.pos = 0 # position pointer
        self.code = defaultdict(lambda:0)
        for i,v in enumerate(code):
            self.code[i] = v
        self.rel = 0 # offset value for relative mode
        self.inputs = []
        
    def add_inputs(self,inputs):
        for i in inputs:
            self.inputs.insert(0,i)
    
    def _parse_opcode(self, opcode):
        """
        Parse an opcode, getting the parameter modes and instruction.
        Opcode should be input as integer.
        """
        code = str(opcode)
        l = len(code)
        inst = int(code[-2:])
        modes = []
        for i in range(l-2):
            modes.append(int(code[l-3-i]))
        return inst, modes
    
    def _get_val_idx(self,loc,mode):
        if mode==0: # position mode
            return self.code[loc]
        elif mode==1: # immediate mode
            return loc
        elif mode==2: # relative mode
            return self.rel+self.code[loc]
        
    def _step(self):
        """
        Exit codes: 
           0 : continue execution
           1 : program finished
           2 : waiting for input
        """
        output = None
        exit_code = 0
        pos = self.pos # where are we in the code

        inst, modes = self._parse_opcode(self.code[pos])
        if inst == 99: # terminate
            exit_code = 1
            
        if inst == 1: # add
            modes = modes+(3-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            self.code[self._get_val_idx(pos+3,modes[2])] = v1+v2 
            pos += 4
            
        if inst == 2: # multiply
            modes = modes+(3-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            self.code[self._get_val_idx(pos+3,modes[2])] = v1*v2 
            pos += 4
            
        if inst == 3: # store input
            try:
                modes = modes + (1-len(modes))*[0]
                self.code[self._get_val_idx(pos+1,modes[0])] = self.inputs.pop()
                pos += 2
            except IndexError: # must wait for additional input
                exit_code = 2
                
        if inst == 4: # output value
            modes = modes+(1-len(modes))*[0]
            v = self.code[self._get_val_idx(pos+1,modes[0])]
            output = v
            pos += 2
            
        if inst == 5: # jump-if-true
            modes = modes+(2-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            if v1 > 0:
                pos = v2
            else:
                pos += 3
                
        if inst == 6: # jump-if-false
            modes = modes+(2-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            if v1 == 0:
                pos = v2
            else:
                pos += 3
                
        if inst == 7:
            modes = modes+(3-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            if v1 < v2:
                self.code[self._get_val_idx(pos+3,modes[2])] = 1
            else:
                self.code[self._get_val_idx(pos+3,modes[2])] = 0
            pos += 4
            
        if inst == 8:
            modes = modes+(3-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            v2 = self.code[self._get_val_idx(pos+2,modes[1])]
            if v1 == v2:
                self.code[self._get_val_idx(pos+3,modes[2])] = 1
            else:
                self.code[self._get_val_idx(pos+3,modes[2])] = 0
            pos += 4
            
        if inst == 9: # adjust rel
            modes = modes+(1-len(modes))*[0]
            v1 = self.code[self._get_val_idx(pos+1,modes[0])]
            self.rel += v1
            pos += 2
        self.pos = pos
        return exit_code, output
    
    def _execute_loop(self):
        out_flag = 0
        outputs = []
        while out_flag==0:
            out_flag, output = self._step()
            if output is not None:
                outputs.append(output)
        return outputs

In [92]:
with open("p17_input.txt","r") as f:
    code = [int(c) for c in f.readline().strip().split(",")]
p = Program(code)
outputs = p._execute_loop()
print("".join(map(chr,outputs))) # visualize the area
arr = []
row = []
for c in outputs:
    if c == 10:
        arr.append(row)
        row = []
    else:
        row.append(c)
def get_indices(i,j):
    return [i,i,i-1,i+1,i],[j-1,j+1,j,j,j]
arr = np.array(arr[:-1])
s = 0
for i in range(1,arr.shape[0]-1):
    for j in range(1,arr.shape[1]-1):
        if arr[[i,i,i-1,i+1,i],[j-1,j+1,j,j,j]].sum() == 5*35:
            s += i*j
print("Part 1 answer: {}".format(s))

....................................#######..........
....................................#.....#..........
....................................#.....#..........
....................................#.....#..........
....................................#.....#..........
....................................#.....#..........
....................................#.....#..........
....................................#.....#..........
..................................#########..........
..................................#.#................
............................#########................
............................#.....#..................
............................#.....#..................
............................#.....#..................
..........................#########..................
..........................#.#........................
..........................#.#........................
..........................#.#........................
..........................#.

In [93]:
# for part 2 I'm just going to visually inspect the map and decide the program structure
# might be an interesting way to automate?
p = "A,B,A,C,B,C,B,A,C,B\n"
a = "L,10,L,6,R,10\n"
b = "R,6,R,8,R,8,L,6,R,8\n"
c = "L,10,R,8,R,8,L,10\n"
f = "n\n"
inputs = list(map(ord,p+a+b+c+f))

with open("p17_input.txt","r") as f:
    code = [int(c) for c in f.readline().strip().split(",")]
code[0] = 2
prog = Program(code)
prog.add_inputs(inputs)
outputs = prog._execute_loop()
print("Part 2 answer: {}".format(outputs[-1]))

Part 2 answer: 1234055
